# Autoencoder 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_score, f1_score
import os

# File path and naming convention
directory = '/Users/zxgan/FYP_Kubernetes/Dataset/cleaned_dataset/'
file_prefix = 'node_node_'
file_suffix = '_dataset.csv'
num_files = 50

# Parameter grid for autoencoder tuning
param_grid_ae = {
    'hidden_layer_1': [64, 128, 256],
    'hidden_layer_2': [32, 64, 128],
    'learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [50, 100],
}

# Initialize summary
results_summary = []

# Loop through each dataset
for i in range(num_files):
    file_path = f"{directory}{file_prefix}{i}{file_suffix}"
    print(f"Processing file: {file_path}")
    
    # Load dataset
    data = pd.read_csv(file_path)
    data.drop(columns=['timestamp', 'pod_status_Pending', 'pod_status_Running', 
                       'pod_status_Succeeded', 'pod_status_Failed', 'pod_status_Unknown'], inplace=True)
    numeric_data = data.select_dtypes(include=[np.number])
    
    # Split into training and testing sets
    from sklearn.model_selection import train_test_split
    X_train, X_test = train_test_split(numeric_data, test_size=0.2, random_state=42)
    
    best_accuracy_ae = 0
    best_f1_score_ae = 0
    best_precision_ae = 0
    best_autoencoder = None
    best_params_ae = None
    num_anomalies = 0

    # Train autoencoder for parameter grid
    for params in ParameterGrid(param_grid_ae):
        print(f"Node {i} - Testing parameters: {params}")
        autoencoder = Sequential([
            Dense(params['hidden_layer_1'], activation='relu', input_shape=(X_train.shape[1],)),
            Dense(params['hidden_layer_2'], activation='relu'),
            Dense(16, activation='relu'),
            Dense(params['hidden_layer_2'], activation='relu'),
            Dense(params['hidden_layer_1'], activation='relu'),
            Dense(X_train.shape[1], activation='sigmoid')
        ])
        autoencoder.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mse')
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        history = autoencoder.fit(
            X_train, X_train, epochs=params['epochs'], batch_size=params['batch_size'],
            validation_split=0.2, callbacks=[early_stopping], verbose=0
        )
        
        # Predict and calculate accuracy
        X_test_pred = autoencoder.predict(X_test)
        test_loss = np.mean(np.power(X_test_pred - X_test, 2), axis=1)
        threshold = np.percentile(test_loss, 95)
        y_test_pred = np.where(test_loss > threshold, 0, 1)
        accuracy = accuracy_score(np.ones(len(y_test_pred)), y_test_pred)
        precision = precision_score(np.ones(len(y_test_pred)), y_test_pred)
        f1 = f1_score(np.ones(len(y_test_pred)), y_test_pred)
        
        if accuracy > best_accuracy_ae:
            best_accuracy_ae = accuracy
            best_f1_score_ae = f1
            best_precision_ae = precision
            best_autoencoder = autoencoder
            best_params_ae = params
            num_anomalies = (y_test_pred == 0).sum()  # Count anomalies (where prediction is 0)
    
    print(f"Node {i} - Best Parameters: {best_params_ae}")
    print(f"Node {i} - Best Accuracy: {best_accuracy_ae}")
    print(f"Node {i} - Number of anomalies detected: {num_anomalies}")
    
    # Save the best model
    model_filename = f"/Users/zxgan/FYP_Kubernetes/Autoencoder_Model/Autoencoder_Model_Node_{i}.h5"
    best_autoencoder.save(model_filename)
    print(f"Model saved: {model_filename}")
    
    # Collect results for the node
    results_summary.append({
        "Node": f"Node_{i}",
        "Best Parameters": str(best_params_ae),
        "Accuracy": best_accuracy_ae,
        "Precision": best_precision_ae,
        "F1 Score": best_f1_score_ae,
        "Anomalies Detected": num_anomalies
    })

# Create a DataFrame for the summary
results_df = pd.DataFrame(results_summary)

# Display the summary in an interactive table
fig_summary = go.Figure(data=[go.Table(
    header=dict(values=list(results_df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[results_df[col] for col in results_df.columns],
               fill_color='lavender',
               align='left'))
])
fig_summary.update_layout(title="Autoencoder Accuracy and Anomalies Summary Across Nodes", title_x=0.5)
fig_summary.show()

# Save the summary to a CSV file
summary_csv_path = os.path.join(directory, "/Users/zxgan/FYP_Kubernetes/Accuracy_Summary/autoencoder_accuracy_and_anomalies_summary.csv")
results_df.to_csv(summary_csv_path, index=False)
print(f"Summary saved to: {summary_csv_path}")

Processing file: /Users/zxgan/FYP_Kubernetes/Dataset/cleaned_dataset/node_node_0_dataset.csv
Node 0 - Testing parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
3/3 [==============================] - 0s 631us/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Node 0 - Best Parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
Node 0 - Best Accuracy: 0.9402985074626866
Node 0 - Number of anomalies detected: 4
Model saved: Autoencoder_Model_Node_0.h5
Processing file: /Users/zxgan/FYP_Kubernetes/Dataset/cleaned_dataset/node_node_1_dataset.csv
Node 1 - Testing parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
2/2 [==============================] - 0s 930us/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Node 1 - Best Parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
Node 1 - Best Accuracy: 0.9508196721311475
Node 1 - Number of anomalies detected: 3
Model saved: Autoencoder_Model_Node_1.h5
Processing file: /Users/zxgan/FYP_Kubernetes/Dataset/cleaned_dataset/node_node_2_dataset.csv
Node 2 - Testing parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
2/2 [==============================] - 0s 872us/step


...

Node 47 - Best Parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
Node 47 - Best Accuracy: 0.9365079365079365
Node 47 - Number of anomalies detected: 4
Model saved: Autoencoder_Model_Node_47.h5
Processing file: /Users/zxgan/FYP_Kubernetes/Dataset/cleaned_dataset/node_node_48_dataset.csv
Node 48 - Testing parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
2/2 [==============================] - 0s 818us/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Node 48 - Best Parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
Node 48 - Best Accuracy: 0.9482758620689655
Node 48 - Number of anomalies detected: 3
Model saved: Autoencoder_Model_Node_48.h5
Processing file: /Users/zxgan/FYP_Kubernetes/Dataset/cleaned_dataset/node_node_49_dataset.csv
Node 49 - Testing parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
2/2 [==============================] - 0s 746us/step
Node 49 - Best Parameters: {'batch_size': 32, 'epochs': 50, 'hidden_layer_1': 64, 'hidden_layer_2': 32, 'learning_rate': 0.001}
Node 49 - Best Accuracy: 0.9433962264150944
Node 49 - Number of anomalies detected: 3
Model saved: Autoencoder_Model_Node_49.h5


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Summary saved to: /Users/zxgan/FYP_Kubernetes/Accuracy_Summary/autoencoder_accuracy_and_anomalies_summary.csv
